In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
print(gpu_info)

from datetime import datetime
from functools import partial
from PIL import Image
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import CIFAR10
from torchvision.models import resnet
from tqdm import tqdm
import argparse
import json
import math
import os
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import lightly.models as models
import lightly.loss as loss

Sun Jun 27 19:35:16 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.80       Driver Version: 460.80       CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  TITAN V             Off  | 00000000:3B:00.0 Off |                  N/A |
| 28%   37C    P2   126W / 250W |   4825MiB / 12066MiB |     18%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  TITAN V             Off  | 00000000:5E:00.0 Off |                  N/A |
| 28%   

/home/l/liny/anaconda3/envs/mypy37-2/lib/python3.7/site-packages/pytorch_lightning/metrics/__init__.py:44: LightningDeprecationWarning: `pytorch_lightning.metrics.*` module has been renamed to `torchmetrics.*` and split off to its own package (https://github.com/PyTorchLightning/metrics) since v1.3 and will be removed in v1.5
  "`pytorch_lightning.metrics.*` module has been renamed to `torchmetrics.*` and split off to its own package"


## Create dataloader, model

In [2]:
parser = argparse.ArgumentParser(description='Train SimSiam on Logo2K')

parser.add_argument('-a', '--arch', default='resnet18')

# lr: 0.06 for batch 512 (or 0.03 for batch 256)
parser.add_argument('--lr', '--learning-rate', default=0.06, type=float, metavar='LR', help='initial learning rate', dest='lr')
parser.add_argument('--epochs', default=800, type=int, metavar='N', help='number of total epochs to run')
parser.add_argument('--schedule', default=[300, 600], nargs='*', type=int, help='learning rate schedule (when to drop lr by 10x); does not take effect if --cos is on')
parser.add_argument('--cos', action='store_true', help='use cosine lr schedule')

parser.add_argument('--batch-size', default=256, type=int, metavar='N', help='mini-batch size')
parser.add_argument('--wd', default=5e-4, type=float, metavar='W', help='weight decay')

parser.add_argument('--bn-splits', default=8, type=int, help='simulate multi-gpu behavior of BatchNorm in one gpu; 1 is SyncBatchNorm in multi-gpu')

parser.add_argument('--symmetric', action='store_true', help='use a symmetric loss function that backprops to both crops')

# knn monitor
parser.add_argument('--knn-k', default=200, type=int, help='k in kNN monitor')
parser.add_argument('--knn-t', default=0.1, type=float, help='softmax temperature in kNN monitor; could be different with moco-t')

# utils
parser.add_argument('--resume', default='', 
                    type=str, metavar='PATH', help='path to latest checkpoint (default: none)')
parser.add_argument('--results-dir', default='', type=str, metavar='PATH', help='path to cache (default: none)')

'''
args = parser.parse_args()  # running in command line
'''
args = parser.parse_args('')  # running in ipynb

# set command line arguments here when running in ipynb
args.epochs = 800
args.cos = True
args.schedule = []  # cos in use
args.symmetric = False
if args.results_dir == '':
    args.results_dir = '../output/Logo2K-' + datetime.now().strftime("%Y-%m-%d-%H-%M-%S-moco")

print(args)

Namespace(arch='resnet18', batch_size=256, bn_splits=8, cos=True, epochs=800, knn_k=200, knn_t=0.1, lr=0.06, results_dir='../output/Logo2K-2021-06-27-19-37-06-moco', resume='', schedule=[], symmetric=False, wd=0.0005)


In [33]:
import pickle
from PIL import Image, ImageOps
from typing import List, Union, Callable

class GetLoader(torch.utils.data.Dataset):
    '''Define customized dataset
    Args:
        data_root:
            Path to directory holding the images to load.
        data_list:
            Path to txt file which map images to labels.
        label_dict:
            Dict which converts label in plain text to label in int.
        transform:
            Transformations to apply.
        grayscale:
            Grayscale model/RGB model, default is RGB
    
    '''
    def __init__(self, data_root, data_list, label_dict, transform=None, grayscale=False):
        
        self.transform = transform
        self.data_root = data_root
        self.grayscale = grayscale
        data_list = [x.strip('\n') for x in open(data_list).readlines()]

        with open(label_dict, 'rb') as handle:
            self.label_dict = pickle.load(handle)

        self.classes = list(self.label_dict.keys())

        self.n_data = len(data_list)

        self.img_paths = []
        self.labels = []
        self.targets = []

        for data in data_list:
            image_path = data
            label = image_path.split('/')[0]
            self.img_paths.append(image_path)
            self.labels.append(label)
            self.targets.append(self.label_dict[label])

    def __getitem__(self, index):

        img_path, label= self.img_paths[index], self.labels[index]
        img_path_full = os.path.join(self.data_root, img_path)
        if self.grayscale:
            img = Image.open(img_path_full).convert('L').convert('RGB')
        else:
            img = Image.open(img_path_full).convert('RGB')

        img = ImageOps.expand(img, (
            (max(img.size) - img.size[0]) // 2, (max(img.size) - img.size[1]) // 2,
            (max(img.size) - img.size[0]) // 2, (max(img.size) - img.size[1]) // 2), fill=(255, 255, 255))

        label = self.label_dict[label]
        if self.transform is not None:
            img = self.transform(img)

        return img, label

    def __len__(self):
        return self.n_data


class GetLoaderPair(GetLoader):
    """CIFAR10 Dataset.
    """
    def __getitem__(self, index):
        
        img_path, label = self.img_paths[index], self.labels[index]
        img_path_full = os.path.join(self.data_root, img_path)
        if self.grayscale:
            img = Image.open(img_path_full).convert('L').convert('RGB')
        else:
            img = Image.open(img_path_full).convert('RGB')

        img = ImageOps.expand(img, (
            (max(img.size) - img.size[0]) // 2, (max(img.size) - img.size[1]) // 2,
            (max(img.size) - img.size[0]) // 2, (max(img.size) - img.size[1]) // 2), fill=(255, 255, 255))

        if self.transform is not None:
            im_1 = self.transform(img)
            im_2 = self.transform(img)

        return im_1, im_2

    
train_transform = transforms.Compose([
    transforms.Resize(128),
    transforms.RandomApply([transforms.ColorJitter(0.4, 0.4, 0.4, 0.1)], p=0.8),
    transforms.RandomGrayscale(p=0.2),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])])

test_transform = transforms.Compose([
    transforms.Resize(128),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])])

In [34]:
# data prepare
train_data = GetLoaderPair(data_root='../datasets/logo2k/train/', 
                           data_list='/home/l/liny/ruofan/PhishIntention/src/siamese_retrain/logo2k/train.txt', 
                           label_dict='/home/l/liny/ruofan/PhishIntention/src/siamese_retrain/logo2k/logo2k_labeldict.pkl',
                           transform=train_transform)

train_loader = DataLoader(train_data, batch_size=args.batch_size, shuffle=True, 
                          num_workers=16, pin_memory=True, drop_last=True)

In [35]:
memory_data = GetLoader(data_root='../datasets/logo2k/train/', 
                        data_list='/home/l/liny/ruofan/PhishIntention/src/siamese_retrain/logo2k/train.txt', 
                        label_dict='/home/l/liny/ruofan/PhishIntention/src/siamese_retrain/logo2k/logo2k_labeldict.pkl',
                        transform=test_transform)
memory_loader = DataLoader(memory_data, batch_size=args.batch_size, 
                           shuffle=False, num_workers=16, pin_memory=True)

In [36]:
test_data = GetLoader(data_root='../datasets/logo2k/test/', 
                      data_list='/home/l/liny/ruofan/PhishIntention/src/siamese_retrain/logo2k/test.txt', 
                      label_dict='/home/l/liny/ruofan/PhishIntention/src/siamese_retrain/logo2k/logo2k_labeldict.pkl',
                      transform=test_transform)

test_loader = DataLoader(test_data, batch_size=args.batch_size, 
                         shuffle=False, num_workers=16, pin_memory=True)

## Model

In [11]:
# use a resnet50 backbone
resnet = torchvision.models.resnet.resnet18()
resnet = torch.nn.Sequential(*list(resnet.children())[:-1])

# build the simsiam model
model = models.SimSiam(resnet, num_ftrs=512)
model = model.cuda()

# use a criterion for self-supervised learning
criterion = loss.SymNegCosineSimilarityLoss()

In [12]:
# train for one epoch
def train(net, data_loader, train_optimizer, epoch, args):
    net.train()
    adjust_learning_rate(optimizer, epoch, args)

    total_loss, total_num, train_bar = 0.0, 0, tqdm(data_loader)
    for im_1, im_2 in train_bar:
        im_1, im_2 = im_1.cuda(non_blocking=True), im_2.cuda(non_blocking=True)

        y0, y1 = net(im_1, im_2)
        # backpropagation
        loss = criterion(y0, y1)
        
        train_optimizer.zero_grad()
        loss.backward()
        train_optimizer.step()

        total_num += data_loader.batch_size
        total_loss += loss.item() * data_loader.batch_size
        train_bar.set_description('Train Epoch: [{}/{}], lr: {:.6f}, Loss: {:.4f}'.format(epoch, args.epochs, optimizer.param_groups[0]['lr'], total_loss / total_num))

    return total_loss / total_num

# lr scheduler for training
def adjust_learning_rate(optimizer, epoch, args):
    """Decay the learning rate based on schedule"""
    lr = args.lr
    if args.cos:  # cosine lr schedule
        lr *= 0.5 * (1. + math.cos(math.pi * epoch / args.epochs))
    else:  # stepwise lr schedule
        for milestone in args.schedule:
            lr *= 0.1 if epoch >= milestone else 1.
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

In [29]:
# test using a knn monitor
def test(net, memory_data_loader, test_data_loader, epoch, args):
    net.eval()
    classes = len(memory_data_loader.dataset.classes)
    total_top1, total_top5, total_num, feature_bank = 0.0, 0.0, 0, []
    with torch.no_grad():
        # generate feature bank
        for data, target in tqdm(memory_data_loader, desc='Feature extracting'):
            feature = net(data.cuda(non_blocking=True)).squeeze()
            feature = F.normalize(feature, dim=1)
            feature_bank.append(feature)
        # [D, N]
        feature_bank = torch.cat(feature_bank, dim=0).t().contiguous()
        # [N]
        feature_labels = torch.tensor(memory_data_loader.dataset.targets, device=feature_bank.device)
        
        # loop test data to predict the label by weighted knn search
        test_bar = tqdm(test_data_loader)
        for data, target in test_bar:
            data, target = data.cuda(non_blocking=True), target.cuda(non_blocking=True)
            feature = net(data).squeeze()
            feature = F.normalize(feature, dim=1)
            
            pred_labels = knn_predict(feature, feature_bank, feature_labels, classes, args.knn_k, args.knn_t)

            total_num += data.size(0)
            total_top1 += (pred_labels[:, 0] == target).float().sum().item()
            test_bar.set_description('Test Epoch: [{}/{}] Acc@1:{:.2f}%'.format(epoch, args.epochs, total_top1 / total_num * 100))

    return total_top1 / total_num * 100

# knn monitor as in InstDisc https://arxiv.org/abs/1805.01978
# implementation follows http://github.com/zhirongw/lemniscate.pytorch and https://github.com/leftthomas/SimCLR
def knn_predict(feature, feature_bank, feature_labels, classes, knn_k, knn_t):
    # compute cos similarity between each feature vector and feature bank ---> [B, N]
    sim_matrix = torch.mm(feature, feature_bank)
    # [B, K]
    sim_weight, sim_indices = sim_matrix.topk(k=knn_k, dim=-1)
    # [B, K]
    sim_labels = torch.gather(feature_labels.expand(feature.size(0), -1), dim=-1, index=sim_indices)
    sim_weight = (sim_weight / knn_t).exp()

    # counts for each class
    one_hot_label = torch.zeros(feature.size(0) * knn_k, classes, device=sim_labels.device)
    # [B*K, C]
    one_hot_label = one_hot_label.scatter(dim=-1, index=sim_labels.view(-1, 1), value=1.0)
    # weighted score ---> [B, C]
    pred_scores = torch.sum(one_hot_label.view(feature.size(0), -1, classes) * sim_weight.unsqueeze(dim=-1), dim=1)

    pred_labels = pred_scores.argsort(dim=-1, descending=True)
    return pred_labels

In [30]:
# define optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=args.lr, weight_decay=args.wd, momentum=0.9)

# load model if resume
epoch_start = 1
if args.resume is not '':
    checkpoint = torch.load(args.resume)
    model.load_state_dict(checkpoint['state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer'])
    epoch_start = checkpoint['epoch'] + 1
    print('Loaded from: {}'.format(args.resume))

In [31]:
# logging
results = {'train_loss': [], 'test_acc@1': []}
if not os.path.exists(args.results_dir):
    os.mkdir(args.results_dir)
    
# dump args
with open(args.results_dir + '/args.json', 'w') as fid:
    json.dump(args.__dict__, fid, indent=2)


In [ ]:
# training loop
for epoch in range(epoch_start, args.epochs + 1):
    test_acc_1 = test(model.backbone, memory_loader, test_loader, epoch, args)
    train_loss = train(model, train_loader, optimizer, epoch, args)
    results['train_loss'].append(train_loss)
    test_acc_1 = test(model.backbone, memory_loader, test_loader, epoch, args)
    results['test_acc@1'].append(test_acc_1)
    # save statistics
    data_frame = pd.DataFrame(data=results, index=range(epoch_start, epoch + 1))
    data_frame.to_csv(args.results_dir + '/log.csv', index_label='epoch')
    # save model
    torch.save({'epoch': epoch, 'state_dict': model.state_dict(), 'optimizer' : optimizer.state_dict(),}, args.results_dir + '/model_last.pth')

Test Epoch: [1/800] Acc@1:9.14%: 100%|██████████| 197/197 [00:27<00:00,  7.07it/s]
Train Epoch: [1/800], lr: 0.060000, Loss: -0.8348: 100%|██████████| 456/456 [03:22<00:00,  2.25it/s]
Test Epoch: [2/800] Acc@1:6.68%: 100%|██████████| 197/197 [00:25<00:00,  7.71it/s]
Train Epoch: [2/800], lr: 0.059999, Loss: -0.9505: 100%|██████████| 456/456 [02:50<00:00,  2.67it/s]
Test Epoch: [3/800] Acc@1:6.35%: 100%|██████████| 197/197 [00:31<00:00,  6.33it/s]
Train Epoch: [3/800], lr: 0.059998, Loss: -0.9602: 100%|██████████| 456/456 [03:01<00:00,  2.51it/s]
Test Epoch: [4/800] Acc@1:6.48%: 100%|██████████| 197/197 [00:35<00:00,  5.52it/s]
Train Epoch: [4/800], lr: 0.059996, Loss: -0.9601: 100%|██████████| 456/456 [03:07<00:00,  2.44it/s]
Test Epoch: [6/800] Acc@1:6.66%: 100%|██████████| 197/197 [00:28<00:00,  6.83it/s]
Train Epoch: [6/800], lr: 0.059992, Loss: -0.9159: 100%|██████████| 456/456 [03:12<00:00,  2.37it/s]
Test Epoch: [7/800] Acc@1:6.32%: 100%|██████████| 197/197 [00:26<00:00,  7.32it/

Test Epoch: [22/800] Acc@1:4.85%: 100%|██████████| 197/197 [00:33<00:00,  5.86it/s]
Train Epoch: [22/800], lr: 0.059888, Loss: -0.9739: 100%|██████████| 456/456 [02:50<00:00,  2.68it/s]
Test Epoch: [23/800] Acc@1:4.92%: 100%|██████████| 197/197 [00:29<00:00,  6.79it/s]
Train Epoch: [23/800], lr: 0.059878, Loss: -0.9829: 100%|██████████| 456/456 [03:14<00:00,  2.35it/s]
Test Epoch: [24/800] Acc@1:4.89%: 100%|██████████| 197/197 [00:33<00:00,  5.86it/s]
Train Epoch: [24/800], lr: 0.059867, Loss: -0.9835: 100%|██████████| 456/456 [02:59<00:00,  2.54it/s]
Test Epoch: [25/800] Acc@1:4.75%: 100%|██████████| 197/197 [00:34<00:00,  5.67it/s]
Train Epoch: [25/800], lr: 0.059856, Loss: -0.9775: 100%|██████████| 456/456 [02:55<00:00,  2.60it/s]
Test Epoch: [26/800] Acc@1:4.73%: 100%|██████████| 197/197 [00:34<00:00,  5.76it/s]
Train Epoch: [26/800], lr: 0.059844, Loss: -0.9883: 100%|██████████| 456/456 [03:12<00:00,  2.37it/s]
Test Epoch: [27/800] Acc@1:4.48%: 100%|██████████| 197/197 [00:36<00:0

Test Epoch: [43/800] Acc@1:3.87%: 100%|██████████| 197/197 [00:42<00:00,  4.62it/s]
Train Epoch: [43/800], lr: 0.059573, Loss: -0.8817: 100%|██████████| 456/456 [03:08<00:00,  2.41it/s]
Test Epoch: [44/800] Acc@1:2.73%: 100%|██████████| 197/197 [00:41<00:00,  4.73it/s]
Train Epoch: [44/800], lr: 0.059553, Loss: -0.9065: 100%|██████████| 456/456 [03:15<00:00,  2.33it/s]
Test Epoch: [45/800] Acc@1:2.82%: 100%|██████████| 197/197 [00:28<00:00,  6.80it/s]
Train Epoch: [45/800], lr: 0.059533, Loss: -0.9105: 100%|██████████| 456/456 [03:01<00:00,  2.51it/s]
Test Epoch: [46/800] Acc@1:2.78%: 100%|██████████| 197/197 [00:37<00:00,  5.23it/s]
Train Epoch: [46/800], lr: 0.059512, Loss: -0.9167: 100%|██████████| 456/456 [03:16<00:00,  2.32it/s]
Test Epoch: [47/800] Acc@1:2.99%: 100%|██████████| 197/197 [00:30<00:00,  6.39it/s]
Train Epoch: [47/800], lr: 0.059490, Loss: -0.9225: 100%|██████████| 456/456 [02:51<00:00,  2.66it/s]
Test Epoch: [48/800] Acc@1:2.67%: 100%|██████████| 197/197 [00:35<00:0

Test Epoch: [63/800] Acc@1:2.60%: 100%|██████████| 197/197 [00:28<00:00,  6.98it/s]
Train Epoch: [63/800], lr: 0.059087, Loss: -0.9052: 100%|██████████| 456/456 [03:22<00:00,  2.26it/s]
Test Epoch: [64/800] Acc@1:2.33%: 100%|██████████| 197/197 [00:33<00:00,  5.93it/s]
Train Epoch: [64/800], lr: 0.059057, Loss: -0.9112: 100%|██████████| 456/456 [03:30<00:00,  2.17it/s]
Test Epoch: [65/800] Acc@1:2.44%: 100%|██████████| 197/197 [00:40<00:00,  4.84it/s]
Train Epoch: [65/800], lr: 0.059028, Loss: -0.9108: 100%|██████████| 456/456 [03:07<00:00,  2.44it/s]
Test Epoch: [66/800] Acc@1:2.56%: 100%|██████████| 197/197 [00:36<00:00,  5.37it/s]
Train Epoch: [66/800], lr: 0.058998, Loss: -0.9192: 100%|██████████| 456/456 [03:02<00:00,  2.51it/s]
Test Epoch: [67/800] Acc@1:2.40%: 100%|██████████| 197/197 [00:27<00:00,  7.06it/s]
Train Epoch: [67/800], lr: 0.058968, Loss: -0.9341: 100%|██████████| 456/456 [03:09<00:00,  2.41it/s]
Test Epoch: [68/800] Acc@1:2.25%: 100%|██████████| 197/197 [00:29<00:0

Test Epoch: [83/800] Acc@1:2.73%: 100%|██████████| 197/197 [00:28<00:00,  6.85it/s]
Train Epoch: [83/800], lr: 0.058421, Loss: -0.9469: 100%|██████████| 456/456 [03:01<00:00,  2.52it/s]
Test Epoch: [84/800] Acc@1:2.85%: 100%|██████████| 197/197 [00:33<00:00,  5.85it/s]
Train Epoch: [84/800], lr: 0.058383, Loss: -0.9572: 100%|██████████| 456/456 [02:47<00:00,  2.72it/s]
Test Epoch: [85/800] Acc@1:2.88%: 100%|██████████| 197/197 [00:29<00:00,  6.67it/s]
Train Epoch: [85/800], lr: 0.058344, Loss: -0.9600: 100%|██████████| 456/456 [02:54<00:00,  2.62it/s]
Test Epoch: [86/800] Acc@1:2.76%: 100%|██████████| 197/197 [00:33<00:00,  5.79it/s]
Train Epoch: [86/800], lr: 0.058305, Loss: -0.9627: 100%|██████████| 456/456 [02:44<00:00,  2.78it/s]
Test Epoch: [87/800] Acc@1:2.76%: 100%|██████████| 197/197 [00:29<00:00,  6.64it/s]
Train Epoch: [87/800], lr: 0.058266, Loss: -0.9055: 100%|██████████| 456/456 [03:06<00:00,  2.45it/s]
Test Epoch: [88/800] Acc@1:2.79%: 100%|██████████| 197/197 [00:29<00:0

Train Epoch: [102/800], lr: 0.057625, Loss: -0.9566: 100%|██████████| 456/456 [02:33<00:00,  2.98it/s]
Test Epoch: [103/800] Acc@1:3.06%: 100%|██████████| 197/197 [00:26<00:00,  7.54it/s]
Train Epoch: [103/800], lr: 0.057579, Loss: -0.9608: 100%|██████████| 456/456 [02:54<00:00,  2.61it/s]
Test Epoch: [104/800] Acc@1:3.16%: 100%|██████████| 197/197 [00:31<00:00,  6.20it/s]
Train Epoch: [104/800], lr: 0.057533, Loss: -0.9504: 100%|██████████| 456/456 [02:35<00:00,  2.93it/s]
Test Epoch: [105/800] Acc@1:3.04%: 100%|██████████| 197/197 [00:25<00:00,  7.83it/s]
Train Epoch: [105/800], lr: 0.057486, Loss: -0.9629: 100%|██████████| 456/456 [02:36<00:00,  2.92it/s]
Test Epoch: [106/800] Acc@1:3.37%: 100%|██████████| 197/197 [00:37<00:00,  5.21it/s]
Train Epoch: [106/800], lr: 0.057438, Loss: -0.9291: 100%|██████████| 456/456 [02:40<00:00,  2.84it/s]
Test Epoch: [107/800] Acc@1:2.72%: 100%|██████████| 197/197 [00:26<00:00,  7.52it/s]
Train Epoch: [107/800], lr: 0.057390, Loss: -0.9474:  32%|██